<a href="https://colab.research.google.com/github/Akashpandey1507/Hotel-Booking-with-PySpark/blob/main/Hotel_Booking_Analysis_with_Py_Spark_EDA_Analsis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hotel Booking Demand with Economic Indicators

To download the datasets and all project for [Click Here](https://github.com/Akashpandey1507/Hotel-Booking-with-PySpark/)

# About Dataset
* This dataset contains 119390 observations for a City Hotel and a Resort Hotel. Each observation represents a hotel booking between the 1st of July 2015 and 31st of August 2017. I have coupled each month of booking with leading economic indicators for the United States for that timeframe using the FRED.

* While this dataset does not specify the location of the hotels, I have gone with the assumption of both being located in the United States for ease of applying economic data for learning purposes

# Acknowledgments
* The data is originally from the article Hotel Booking Demand Datasets, written by Nuno Antonio, Ana Almeida, and Luis Nunes for Data in Brief, Volume 22, February 2019. The original cleaned version was uploaded to Kaggle by by Thomas Mock and Antoine Bichat. I used that dataset to couple with various indicators:

* GDP
* CPI
* CPI for Hotels
* Fuel Prices
* Disposable income per capita
* Inflation
* Consumer Sentiment
* GINI Indicator

In [1]:
!pip install pyspark
#installing the pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=73f13db5d8d5aabab3fb79787c904bf5468cd79b05803ebb5abc0d941a69bcef
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
# Importing libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
spark = SparkSession.builder.appName("Hotel Booking").getOrCreate()
# Create the spark session

In [4]:
file = "/content/drive/MyDrive/Data Analysis Files/hotel_bookings_raw.csv"

# File Path

In [5]:
original_datasets = spark.read.csv(
    file,
    header=True,
    inferSchema=True
)

# Loading the datasets

In [6]:
df = original_datasets.alias("copy")
# Create the duplicate datasets of original

In [7]:
df.show() # show the datasets

+------------+-----------+---------+-----------------+------------------+------------------------+-------------------------+-----------------------+--------------------+------+--------+------+----+-------+--------------+--------------------+-----------------+----------------------+------------------------------+------------------+------------------+---------------+------------+-----+--------------------+-------------+------+---------------------------+-------------------------+------------------+-----------------------+------+-------+---------+-------------+---------+------+-------+---------+---------+-----------+-------+-------+
|       hotel|is_canceled|lead_time|arrival_date_year|arrival_date_month|arrival_date_week_number|arrival_date_day_of_month|stays_in_weekend_nights|stays_in_week_nights|adults|children|babies|meal|country|market_segment|distribution_channel|is_repeated_guest|previous_cancellations|previous_bookings_not_canceled|reserved_room_type|assigned_room_type|booking_cha

# Data Processing

In [8]:
df.count() # check the len of datasets

119390

In [9]:
df.columns # show all columns

['hotel',
 'is_canceled',
 'lead_time',
 'arrival_date_year',
 'arrival_date_month',
 'arrival_date_week_number',
 'arrival_date_day_of_month',
 'stays_in_weekend_nights',
 'stays_in_week_nights',
 'adults',
 'children',
 'babies',
 'meal',
 'country',
 'market_segment',
 'distribution_channel',
 'is_repeated_guest',
 'previous_cancellations',
 'previous_bookings_not_canceled',
 'reserved_room_type',
 'assigned_room_type',
 'booking_changes',
 'deposit_type',
 'agent',
 'days_in_waiting_list',
 'customer_type',
 'adr',
 'required_car_parking_spaces',
 'total_of_special_requests',
 'reservation_status',
 'reservation_status_date',
 'MO_YR',
 'CPI_AVG',
 'INFLATION',
 'INFLATION_CHG',
 'CSMR_SENT',
 'UNRATE',
 'INTRSRT',
 'GDP',
 'FUEL_PRCS',
 'CPI_HOTELS',
 'US_GINI',
 'DIS_INC']

In [10]:
len(df.columns) # check the number of columns

43

In [11]:
print(f"The Datasets is having {df.count()} rows and {len(df.columns)} columns.")

The Datasets is having 119390 rows and 43 columns.


In [12]:
df.select([f.sum(f.col(i).isNull().cast('int')).alias(i) for i in df.columns]).show() # checking the null values

+-----+-----------+---------+-----------------+------------------+------------------------+-------------------------+-----------------------+--------------------+------+--------+------+----+-------+--------------+--------------------+-----------------+----------------------+------------------------------+------------------+------------------+---------------+------------+-----+--------------------+-------------+---+---------------------------+-------------------------+------------------+-----------------------+-----+-------+---------+-------------+---------+------+-------+---+---------+----------+-------+-------+
|hotel|is_canceled|lead_time|arrival_date_year|arrival_date_month|arrival_date_week_number|arrival_date_day_of_month|stays_in_weekend_nights|stays_in_week_nights|adults|children|babies|meal|country|market_segment|distribution_channel|is_repeated_guest|previous_cancellations|previous_bookings_not_canceled|reserved_room_type|assigned_room_type|booking_changes|deposit_type|agent|d

In [13]:
df.printSchema() # checking the datatypes opf each columns of the datasets

root
 |-- hotel: string (nullable = true)
 |-- is_canceled: integer (nullable = true)
 |-- lead_time: integer (nullable = true)
 |-- arrival_date_year: integer (nullable = true)
 |-- arrival_date_month: string (nullable = true)
 |-- arrival_date_week_number: integer (nullable = true)
 |-- arrival_date_day_of_month: integer (nullable = true)
 |-- stays_in_weekend_nights: integer (nullable = true)
 |-- stays_in_week_nights: integer (nullable = true)
 |-- adults: integer (nullable = true)
 |-- children: string (nullable = true)
 |-- babies: integer (nullable = true)
 |-- meal: string (nullable = true)
 |-- country: string (nullable = true)
 |-- market_segment: string (nullable = true)
 |-- distribution_channel: string (nullable = true)
 |-- is_repeated_guest: integer (nullable = true)
 |-- previous_cancellations: integer (nullable = true)
 |-- previous_bookings_not_canceled: integer (nullable = true)
 |-- reserved_room_type: string (nullable = true)
 |-- assigned_room_type: string (nullab

In [14]:
[i for i,v in df.dtypes if v in ['string']]
# filter the those columns where datatypppe is string object

['hotel',
 'arrival_date_month',
 'children',
 'meal',
 'country',
 'market_segment',
 'distribution_channel',
 'reserved_room_type',
 'assigned_room_type',
 'deposit_type',
 'agent',
 'customer_type',
 'reservation_status',
 'reservation_status_date',
 'MO_YR',
 'CPI_AVG',
 'INFLATION',
 'INFLATION_CHG',
 'CSMR_SENT',
 'UNRATE',
 'INTRSRT',
 'GDP',
 'FUEL_PRCS',
 'CPI_HOTELS',
 'US_GINI',
 'DIS_INC']

In [15]:
df.select([i for i,v in df.dtypes if v in ['string']]).show()
# show the filtered the those columns where datatypppe is string object

+------------+------------------+--------+----+-------+--------------+--------------------+------------------+------------------+------------+-----+-------------+------------------+-----------------------+------+-------+---------+-------------+---------+------+-------+---------+---------+-----------+-------+-------+
|       hotel|arrival_date_month|children|meal|country|market_segment|distribution_channel|reserved_room_type|assigned_room_type|deposit_type|agent|customer_type|reservation_status|reservation_status_date| MO_YR|CPI_AVG|INFLATION|INFLATION_CHG|CSMR_SENT|UNRATE|INTRSRT|      GDP|FUEL_PRCS| CPI_HOTELS|US_GINI|DIS_INC|
+------------+------------------+--------+----+-------+--------------+--------------------+------------------+------------------+------------+-----+-------------+------------------+-----------------------+------+-------+---------+-------------+---------+------+-------+---------+---------+-----------+-------+-------+
|Resort Hotel|              July|       0|  BB

In [16]:
df.groupBy(f.col('CPI_AVG')).count().show()

+-------+-----+
|CPI_AVG|count|
+-------+-----+
|243.892| 4826|
|244.193| 4634|
|244.243| 4038|
|238.034| 3615|
|237.733| 5742|
|237.652| 4482|
|237.001|  275|
|244.006| 4858|
|238.017| 3077|
|237.761| 3062|
|245.183| 3417|
|236.222|  151|
|235.342|   44|
|240.222| 4552|
|244.004| 5006|
|242.026| 5021|
|244.163| 4060|
|241.176| 4993|
|242.637| 4338|
|246.435|  393|
+-------+-----+
only showing top 20 rows



In [17]:
df = df.withColumn("CPI_AVG", f.col("CPI_AVG").cast("double"))
df = df.withColumn("INFLATION_CHG", f.col("INFLATION_CHG").cast("double"))
df = df.withColumn("CSMR_SENT", f.col("CSMR_SENT").cast("double"))
df = df.withColumn("UNRATE", f.col("UNRATE").cast("double"))
df = df.withColumn("INTRSRT", f.col("INTRSRT").cast("double"))
df = df.withColumn("GDP", f.col("GDP").cast("double"))
df = df.withColumn("FUEL_PRCS", f.col("FUEL_PRCS").cast("double"))
df = df.withColumn("CPI_HOTELS", f.col("CPI_HOTELS").cast("double"))
df = df.withColumn("US_GINI", f.col("US_GINI").cast("double"))
df = df.withColumn("DIS_INC", f.col("DIS_INC").cast("double"))
df = df.withColumn("INFLATION", f.col("INFLATION").cast("double"))
df = df.withColumn("children", f.col("children").cast("double"))
df = df.withColumn("agent", (f.when(f.col("agent")== "NULL", "0").otherwise(f.col("agent"))).cast('double'))

In [18]:
[i for i,v in df.dtypes if v in ['string']]

['hotel',
 'arrival_date_month',
 'meal',
 'country',
 'market_segment',
 'distribution_channel',
 'reserved_room_type',
 'assigned_room_type',
 'deposit_type',
 'customer_type',
 'reservation_status',
 'reservation_status_date',
 'MO_YR']

In [19]:
df.select([i for i,v in df.dtypes if v in ['string']]).show()

+------------+------------------+----+-------+--------------+--------------------+------------------+------------------+------------+-------------+------------------+-----------------------+------+
|       hotel|arrival_date_month|meal|country|market_segment|distribution_channel|reserved_room_type|assigned_room_type|deposit_type|customer_type|reservation_status|reservation_status_date| MO_YR|
+------------+------------------+----+-------+--------------+--------------------+------------------+------------------+------------+-------------+------------------+-----------------------+------+
|Resort Hotel|              July|  BB|    PRT|        Direct|              Direct|                 C|                 C|  No Deposit|    Transient|         Check-Out|               7/1/2015|7-2015|
|Resort Hotel|              July|  BB|    PRT|        Direct|              Direct|                 C|                 C|  No Deposit|    Transient|         Check-Out|               7/1/2015|7-2015|
|Resort Ho

In [20]:
from pyspark.sql import types as t

In [21]:
[i for i,v in df.dtypes if v in ['string']]

['hotel',
 'arrival_date_month',
 'meal',
 'country',
 'market_segment',
 'distribution_channel',
 'reserved_room_type',
 'assigned_room_type',
 'deposit_type',
 'customer_type',
 'reservation_status',
 'reservation_status_date',
 'MO_YR']

In [22]:
df.select([i for i,v in df.dtypes if v in ['string']]).show()

+------------+------------------+----+-------+--------------+--------------------+------------------+------------------+------------+-------------+------------------+-----------------------+------+
|       hotel|arrival_date_month|meal|country|market_segment|distribution_channel|reserved_room_type|assigned_room_type|deposit_type|customer_type|reservation_status|reservation_status_date| MO_YR|
+------------+------------------+----+-------+--------------+--------------------+------------------+------------------+------------+-------------+------------------+-----------------------+------+
|Resort Hotel|              July|  BB|    PRT|        Direct|              Direct|                 C|                 C|  No Deposit|    Transient|         Check-Out|               7/1/2015|7-2015|
|Resort Hotel|              July|  BB|    PRT|        Direct|              Direct|                 C|                 C|  No Deposit|    Transient|         Check-Out|               7/1/2015|7-2015|
|Resort Ho

In [23]:
df.select("reservation_status_date").show()

+-----------------------+
|reservation_status_date|
+-----------------------+
|               7/1/2015|
|               7/1/2015|
|               7/2/2015|
|               7/2/2015|
|               7/3/2015|
|               7/3/2015|
|               7/3/2015|
|               7/3/2015|
|               5/6/2015|
|              4/22/2015|
|              6/23/2015|
|               7/5/2015|
|               7/5/2015|
|               7/5/2015|
|               7/5/2015|
|               7/5/2015|
|               7/5/2015|
|               7/2/2015|
|               7/2/2015|
|               7/5/2015|
+-----------------------+
only showing top 20 rows



In [24]:
df.groupBy(f.col("reservation_status_date")).count().show()

+-----------------------+-----+
|reservation_status_date|count|
+-----------------------+-----+
|             11/14/2015|   80|
|              6/29/2016|  113|
|              7/31/2016|  125|
|              8/21/2016|  139|
|              12/8/2016|  153|
|              3/20/2017|  194|
|              2/27/2015|    1|
|              8/31/2015|  116|
|               8/9/2016|  121|
|             11/24/2016|  172|
|               5/7/2017|  146|
|             11/26/2015|  135|
|              8/28/2016|  134|
|              2/14/2017|  121|
|              4/10/2017|  164|
|              6/11/2016|   89|
|              8/31/2016|  155|
|             11/30/2016|  137|
|              7/20/2017|  104|
|              6/15/2015|   53|
+-----------------------+-----+
only showing top 20 rows



In [25]:
df = df.withColumn("reservation_status_date", f.to_date(df["reservation_status_date"], "M/d/yyyy"))

In [26]:
df.select("reservation_status_date").show()

+-----------------------+
|reservation_status_date|
+-----------------------+
|             2015-07-01|
|             2015-07-01|
|             2015-07-02|
|             2015-07-02|
|             2015-07-03|
|             2015-07-03|
|             2015-07-03|
|             2015-07-03|
|             2015-05-06|
|             2015-04-22|
|             2015-06-23|
|             2015-07-05|
|             2015-07-05|
|             2015-07-05|
|             2015-07-05|
|             2015-07-05|
|             2015-07-05|
|             2015-07-02|
|             2015-07-02|
|             2015-07-05|
+-----------------------+
only showing top 20 rows



In [27]:
[i for i,v in df.dtypes if v in ['string']]

['hotel',
 'arrival_date_month',
 'meal',
 'country',
 'market_segment',
 'distribution_channel',
 'reserved_room_type',
 'assigned_room_type',
 'deposit_type',
 'customer_type',
 'reservation_status',
 'MO_YR']

In [28]:
df.select([i for i,v in df.dtypes if v in ['string']]).show()

+------------+------------------+----+-------+--------------+--------------------+------------------+------------------+------------+-------------+------------------+------+
|       hotel|arrival_date_month|meal|country|market_segment|distribution_channel|reserved_room_type|assigned_room_type|deposit_type|customer_type|reservation_status| MO_YR|
+------------+------------------+----+-------+--------------+--------------------+------------------+------------------+------------+-------------+------------------+------+
|Resort Hotel|              July|  BB|    PRT|        Direct|              Direct|                 C|                 C|  No Deposit|    Transient|         Check-Out|7-2015|
|Resort Hotel|              July|  BB|    PRT|        Direct|              Direct|                 C|                 C|  No Deposit|    Transient|         Check-Out|7-2015|
|Resort Hotel|              July|  BB|    GBR|        Direct|              Direct|                 A|                 C|  No Depos

In [29]:
[i for i,v in df.dtypes if v in ['int', 'double']]

['is_canceled',
 'lead_time',
 'arrival_date_year',
 'arrival_date_week_number',
 'arrival_date_day_of_month',
 'stays_in_weekend_nights',
 'stays_in_week_nights',
 'adults',
 'children',
 'babies',
 'is_repeated_guest',
 'previous_cancellations',
 'previous_bookings_not_canceled',
 'booking_changes',
 'agent',
 'days_in_waiting_list',
 'adr',
 'required_car_parking_spaces',
 'total_of_special_requests',
 'CPI_AVG',
 'INFLATION',
 'INFLATION_CHG',
 'CSMR_SENT',
 'UNRATE',
 'INTRSRT',
 'GDP',
 'FUEL_PRCS',
 'CPI_HOTELS',
 'US_GINI',
 'DIS_INC']

In [30]:
df.select([i for i,v in df.dtypes if v in ['int', 'double']]).show()

+-----------+---------+-----------------+------------------------+-------------------------+-----------------------+--------------------+------+--------+------+-----------------+----------------------+------------------------------+---------------+-----+--------------------+------+---------------------------+-------------------------+-------+---------+-------------+---------+------+-------+---------+---------+-----------+-------+-------+
|is_canceled|lead_time|arrival_date_year|arrival_date_week_number|arrival_date_day_of_month|stays_in_weekend_nights|stays_in_week_nights|adults|children|babies|is_repeated_guest|previous_cancellations|previous_bookings_not_canceled|booking_changes|agent|days_in_waiting_list|   adr|required_car_parking_spaces|total_of_special_requests|CPI_AVG|INFLATION|INFLATION_CHG|CSMR_SENT|UNRATE|INTRSRT|      GDP|FUEL_PRCS| CPI_HOTELS|US_GINI|DIS_INC|
+-----------+---------+-----------------+------------------------+-------------------------+-----------------------+

In [31]:
[i for i,v in df.dtypes if v in ['date']]

['reservation_status_date']

In [32]:
df.select([i for i,v in df.dtypes if v in ['date']]).show()

+-----------------------+
|reservation_status_date|
+-----------------------+
|             2015-07-01|
|             2015-07-01|
|             2015-07-02|
|             2015-07-02|
|             2015-07-03|
|             2015-07-03|
|             2015-07-03|
|             2015-07-03|
|             2015-05-06|
|             2015-04-22|
|             2015-06-23|
|             2015-07-05|
|             2015-07-05|
|             2015-07-05|
|             2015-07-05|
|             2015-07-05|
|             2015-07-05|
|             2015-07-02|
|             2015-07-02|
|             2015-07-05|
+-----------------------+
only showing top 20 rows



In [33]:
df.printSchema()

root
 |-- hotel: string (nullable = true)
 |-- is_canceled: integer (nullable = true)
 |-- lead_time: integer (nullable = true)
 |-- arrival_date_year: integer (nullable = true)
 |-- arrival_date_month: string (nullable = true)
 |-- arrival_date_week_number: integer (nullable = true)
 |-- arrival_date_day_of_month: integer (nullable = true)
 |-- stays_in_weekend_nights: integer (nullable = true)
 |-- stays_in_week_nights: integer (nullable = true)
 |-- adults: integer (nullable = true)
 |-- children: double (nullable = true)
 |-- babies: integer (nullable = true)
 |-- meal: string (nullable = true)
 |-- country: string (nullable = true)
 |-- market_segment: string (nullable = true)
 |-- distribution_channel: string (nullable = true)
 |-- is_repeated_guest: integer (nullable = true)
 |-- previous_cancellations: integer (nullable = true)
 |-- previous_bookings_not_canceled: integer (nullable = true)
 |-- reserved_room_type: string (nullable = true)
 |-- assigned_room_type: string (nullab

In [34]:
df.show()

+------------+-----------+---------+-----------------+------------------+------------------------+-------------------------+-----------------------+--------------------+------+--------+------+----+-------+--------------+--------------------+-----------------+----------------------+------------------------------+------------------+------------------+---------------+------------+-----+--------------------+-------------+------+---------------------------+-------------------------+------------------+-----------------------+------+-------+---------+-------------+---------+------+-------+---------+---------+-----------+-------+-------+
|       hotel|is_canceled|lead_time|arrival_date_year|arrival_date_month|arrival_date_week_number|arrival_date_day_of_month|stays_in_weekend_nights|stays_in_week_nights|adults|children|babies|meal|country|market_segment|distribution_channel|is_repeated_guest|previous_cancellations|previous_bookings_not_canceled|reserved_room_type|assigned_room_type|booking_cha